In [9]:
pwd

u'/Users/naveenkumar/Dropbox/OnlineWork/mlusecase/Notebooks/Simility'

In [10]:
%matplotlib inline
import pandas as pd
import os
import dateutil
import warnings
import csv
import numpy as np
import re
import gc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding
import hypertools as hyp

from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

import datetime as dt
import pandas as pd
import numpy as np
import warnings
import string

warnings.filterwarnings('ignore')

wdir = "/Users/naveenkumar/simility/quoka"
LABELS = os.path.join(wdir, "quoka_y.csv")
INSTANCES = os.path.join(wdir, "quoka_uniq_filtered_cols.csv")
#INSTANCES = os.path.join(wdir, "quoka_uniq_sample.csv")
#INSTANCES = os.path.join(wdir, "quoka_uniq.csv")


In [59]:
y_d = pd.read_csv(LABELS,sep=',',header='infer',quotechar='"')


In [12]:
print(y_d['sim_dc'].value_counts())
print(y_d['eid'].nunique())#4506988
print(y_d.shape[0])#4506988
y_d = y_d.sort_values('sim_updated_at', ascending=False).drop_duplicates('eid').sort_index().reset_index(drop=True)
print(y_d.shape[0])

{'OK'}                     3494510
{}                          696849
{'SCORING_DELETE'}          306559
{'OK','SCORING_DELETE'}       9058
{'BLOCKED'}                     11
{'CLARIFY'}                      1
Name: sim_dc, dtype: int64
4506988
4506988
4506988


In [13]:
x_d = pd.read_csv(INSTANCES, sep=',',header='infer',quotechar='"', low_memory=False)
print(x_d.shape)

(358317, 1466)


In [15]:
x_d['eid'] = x_d['eid'].apply(lambda x:str(x))

In [16]:
data = pd.merge(x_d,y_d[['eid','sim_dc']], how='inner', on='eid')
print(data.shape)

(52149, 1467)


In [41]:
data['eid'].head()



0    202893151
1    202893318
2    202893577
3    202893607
4    202893652
Name: eid, dtype: object

In [8]:
del y_d
del x_d
import gc
gc.collect()

46

In [ ]:
data.to_csv(os.path.join(wdir,"quoka_x_y.csv"))

In [25]:
set1 = y_d['eid'].tolist()
set2 = x_d['eid'].tolist()

In [26]:
print(len(set1))
print(len(set2))


358317

In [36]:
diffset = list(set(set2) - set(set1))

In [40]:
len(diffset)

306168

In [51]:
diffset[0]
y_d['eid'].dtype

dtype('O')

In [61]:
from string import strip
y_d = y_d.replace('^\s+','', regex=True)
y_d = y_d.replace('\s+$','', regex=True)

In [62]:

#y_d['eid'] = y_d['eid'].apply(lambda x:strip(x))
y_d[y_d['eid'] == '201738651']

,eid,sim_created_at,sim_creation_date,sim_dc,sim_updated_at


In [10]:
#summarizing the fields

percentiles_list = [0.01, 0.02, 0.05, 0.25, 0.50, 0.75, 0.95, 0.98, 0.99]
x_d_allsummary = x_d.describe(percentiles=percentiles_list, exclude='object').transpose()
x_d_allsummary.reset_index(level=0, inplace=True)
x_d_allsummary.rename(columns={'index': 'ColName'}, inplace=True)
x_d_allsummary['DataType'] = x_d_allsummary['ColName'].apply(lambda col:x_d[col].dtype)
x_d_allsummary['TotalNullValues'] = x_d_allsummary['ColName'].apply(lambda col:x_d[col].isnull().sum())

In [11]:
x_d_allsummary.to_csv(os.path.join(wdir,"all_summary.psv"), sep='|')

In [50]:
drop_col_list = ['ad_text_md5','ad_text_md5_eid','ad_text','ad_text_md5_ad_text','ad_text_1','ad_headline','ad_headline_1','ad_headline_1_eid']
x_d.drop(drop_col_list, axis=1, inplace=True)
x_d.to_csv(os.path.join(wdir,"quoka_sample_1L_uniq.csv"), sep=',', quotechar='~')

In [33]:
ColList = pd.DataFrame(x_d.dtypes, columns=['Dtype'])
ColList['ColName'] = ColList.index
ColList = ColList[['ColName','Dtype']]
ColList.reset_index(drop=True, inplace=True)
ColList['Dtype'].value_counts()

float64    2395
object      165
int64        22
bool          3
Name: Dtype, dtype: int64

In [2]:
collist = pd.read_csv('/Users/naveenkumar/simility/quoka/Col_List.csv')
prefix_str = '^(ad_address|ad_email_address|ad_email_domain|ad_headline_1|ad_image_hash_1|ad_phone_1|ad_place_path|ad_text_email_content|ad_text_email_content_copy|ad_text_md5|ad_text_phone_number_content|city_category|device_id|device_id_ip|session_id|user_address|user_email_address|user_id|user_phone_1|username|username_num|username_variability)_'
time_str = '_(1day|7day|30day|lifetime|1hour|lifetime_count|lifetime_sum)$'
suffix_str = '_per_(ad_address|ad_email_address|ad_email_domain|ad_headline_1|ad_image_hash_1|ad_phone_1|ad_place_path|ad_text_email_content|ad_text_email_content_copy|ad_text_md5|ad_text_phone_number_content|city_category|device_id|device_id_ip|session_id|user_address|user_email_address|user_id|user_phone_1|username|username_num|username_variability)$'


collist['NewColName1'] = collist['ColName'].apply(lambda x:re.sub(prefix_str, '', x))
collist['NewColName2'] = collist['NewColName1'].apply(lambda x:re.sub(time_str, '', x))
collist['NewColName3'] = collist['NewColName2'].apply(lambda x:re.sub(suffix_str, '', x))

print(collist[['NewColName3']].head())
collist.to_csv(os.path.join(wdir,'col_reg.csv'))

t = collist.groupby(by='NewColName3', as_index=False).agg({'Base Table Id': pd.Series.nunique})
t.rename(columns={'Base Table Id' : 'Uniquecat'}, inplace = True)

                NewColName3
0     ad_address_kbdistance
1         ad_address_length
2  ad_address_nonalpharatio
3    ad_address_repeatratio
4     ad_address_vowelratio


In [3]:
print(t['Uniquecat'].value_counts())

1     894
2      20
19      4
3       4
Name: Uniquecat, dtype: int64
